### Day 1: Sonar Sweep

In [69]:
input_file = 'input_day1.txt'
measurements = [int(l.strip()) for l in open(input_file).readlines()]
print(len(measurements))
increases = [measurements[i] > measurements[i-1] for i in range(1, len(measurements))]
print(len(increases), sum(increases))

2000
1999 1393


In [70]:
# Sanity check
for i in range(10):
    print(measurements[i], '\t', (increases[i-1] if i > 0 else ''))

156 	 
176 	 True
175 	 False
176 	 True
183 	 True
157 	 False
150 	 False
153 	 True
154 	 True
170 	 True


In [71]:
measurement_windows = (measurements[i:i+3] for i in range(len(measurements)-2))
measurement_sums = [sum(window) for window in measurement_windows]
print(len(measurement_sums), measurement_sums[:10])

1998 [507, 527, 534, 516, 490, 460, 457, 477, 486, 499]


In [72]:
increases = [measurement_sums[i] > measurement_sums[i-1] for i in range(1, len(measurement_sums))]
print(len(increases), sum(increases))

1997 1359


### Day 2: Dive!

In [73]:
input_file = 'input_day2.txt'
commands = [l.strip().split() for l in  open(input_file).readlines()]
hpos, depth = 0, 0
for command in commands:
    direction, X = command[0], int(command[1])
    if direction == 'up':
        depth -= X
    elif direction == 'down':
        depth += X
    elif direction == 'forward':
        hpos += X
hpos, depth, hpos * depth

(1968, 1063, 2091984)

In [74]:
hpos, depth, aim = 0, 0, 0
for command in commands:
    direction, X = command[0], int(command[1])
    if direction == 'up':
        aim -= X
    elif direction == 'down':
        aim += X
    elif direction == 'forward':
        hpos += X
        depth += aim * X
hpos, depth, hpos * depth

(1968, 1060092, 2086261056)

### Day 3: Binary Diagnostic

In [75]:
input_file = 'input_day3.txt'
binary_nums = [list(map(int, l.strip())) for l in open(input_file).readlines()]
len(binary_nums), binary_nums[0]

(1000, [1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1])

In [76]:
import numpy as np
bit_sums = np.sum(binary_nums, axis=0)
bit_sums

array([511, 497, 496, 506, 504, 519, 487, 488, 499, 520, 503, 499])

In [87]:
most_common_mask = (bit_sums > len(binary_nums) / 2)
gamma = list(map(int, most_common_mask))
epsilon = list(map(int, ~most_common_mask))

def bitlist_to_num(l):
    return int(''.join(map(str, l)), 2)

gamma, epsilon, bitlist_to_num(gamma), bitlist_to_num(epsilon), bitlist_to_num(gamma) * bitlist_to_num(epsilon)

([1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0],
 [0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1],
 2502,
 1593,
 3985686)

In [124]:
def most_common_bit(binary_numbers, pos=0):
    """Returns most common bit at a given position, or 1 if 0 and 1 are present the same number of times."""
    return int(np.mean([num[pos] for num in binary_numbers]) >= 0.5)
    
bin_nums = [
    [0, 1, 0],
    [0, 1, 1],
    [1, 0, 1],
    [0, 1, 0]
]
assert most_common_bit(bin_nums, 0) == 0
assert most_common_bit(bin_nums, 1) == most_common_bit(bin_nums, 2) == 1

In [117]:
def filter_by_criteria(binary_numbers, criteria='most common', pos=0):
    if len(binary_numbers) == 1:
        return int(''.join(map(str, binary_numbers[0])), 2)
    
    bit_filter = most_common_bit(binary_numbers, pos)
    if criteria != 'most common':
        bit_filter = abs(bit_filter - 1)
        
    filtered_numbers = filter(lambda num: num[pos] == bit_filter, binary_numbers)
    return filter_by_criteria(list(filtered_numbers), criteria, pos+1)

In [128]:
# Check provided example
sample = """
00100
11110
10110
10111
10101
01111
00111
11100
10000
11001
00010
01010
""".strip()
sample = [list(map(int, s)) for s in sample.split('\n') if s]
assert filter_by_criteria(sample) == 23  # oxygen generator rating
assert filter_by_criteria(sample, criteria='least common') == 10  # CO2 scrubber rating

In [130]:
filter_by_criteria(binary_nums) * filter_by_criteria(binary_nums, criteria='least common')

2555739

### Day 4: Giant Squid

In [209]:
sample = """
7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7
""".strip().split('\n')

In [242]:
def parse_bingo_input(lines):
    drawn_numbers = list(map(int, lines[0].split(',')))
    boards, board = [], []
    for l in lines[2:]:
        if l.strip():
            board.append(list(map(int, l.strip().split())))
            if len(board) == 5:
                boards.append(board)
                board = []
    return drawn_numbers, boards

parse_bingo_input(sample)

([7,
  4,
  9,
  5,
  11,
  17,
  23,
  2,
  0,
  14,
  21,
  24,
  10,
  16,
  13,
  6,
  15,
  25,
  12,
  22,
  18,
  20,
  8,
  19,
  3,
  26,
  1],
 [[[22, 13, 17, 11, 0],
   [8, 2, 23, 4, 24],
   [21, 9, 14, 16, 7],
   [6, 10, 3, 18, 5],
   [1, 12, 20, 15, 19]],
  [[3, 15, 0, 2, 22],
   [9, 18, 13, 17, 5],
   [19, 8, 7, 25, 23],
   [20, 11, 10, 24, 4],
   [14, 21, 16, 12, 6]],
  [[14, 21, 17, 24, 4],
   [10, 16, 15, 9, 19],
   [18, 8, 23, 26, 20],
   [22, 11, 13, 6, 5],
   [2, 0, 12, 3, 7]]])

In [243]:
class BingoBoard(object):
    def __init__(self, values):
        assert len(values) == 5, values
        assert all(len(v) == 5 for v in values), values
        
        self.values = np.array(values, dtype=int)
        self.drawn = np.zeros((5, 5), dtype=bool)
        self.score = 0
        self.won = False

    def __repr__(self):
        return '\n'.join(
            '\t'.join(
                f'{"*" if self.drawn[i,j] else " "}{val}' 
                for j, val in enumerate(row)
            )
            for i, row in enumerate(self.values)
        )
    
    def draw_number(self, *ns):
        for n in ns:
            self.drawn[self.values==n] = True
            self.won = self._has_won()
            if self.won:
                self.score = n * self.values[~self.drawn].sum()
        
    def _has_won(self):
        return bool(self.drawn.prod(axis=0).sum()) or bool(self.drawn.prod(axis=1).sum())
    
bb = BingoBoard(parse_bingo_input(sample)[1][2])
bb.draw_number(7, 4, 9, 5, 11, 17, 23, 2, 0, 14, 21, 24)
assert bb.won and bb.score == 4512
bb

*14	*21	*17	*24	*4
 10	 16	 15	*9	 19
 18	 8	*23	 26	 20
 22	*11	 13	 6	*5
*2	*0	 12	 3	*7

In [272]:
drawn_numbers, board_values = parse_bingo_input(open('input_day4.txt').readlines())
print(len(drawn_numbers), len(board_values))
board_values[0]

100 100


[[31, 23, 52, 26, 8],
 [27, 89, 37, 80, 46],
 [97, 19, 63, 34, 79],
 [13, 59, 45, 12, 73],
 [42, 25, 22, 6, 39]]

In [273]:
# Part 1: Pick the first winning board
done = False
boards = [BingoBoard(b) for b in board_values]
for n in drawn_numbers:
    print(n, end='.. ')
    for b in boards:
        b.draw_number(n)
        if b.won:
            print(b.score)
            print(b)
            done = True
            break
    if done:
        break

27.. 14.. 70.. 7.. 85.. 66.. 65.. 57.. 68.. 23.. 33.. 78.. 4.. 84.. 25.. 18.. 43.. 71.. 76.. 61.. 34.. 82.. 93.. 74.. 64084
*7	*70	 5	 69	*4
*34	 60	 40	 73	 6
*74	 54	 67	 32	 38
*93	 62	 17	 51	 86
*57	 88	 99	 3	 16


In [274]:
# Part 2: Pick the last winning board
boards = [BingoBoard(b) for b in board_values]
boards_done = []
for n in drawn_numbers:
    print(n, end='.. ')
    for i, b in enumerate(boards):
        if i in boards_done:
            continue
            
        b.draw_number(n)
        if b.won:
            # print(f'Board {i} won!')
            boards_done.append(i)
            if len(boards) == len(boards_done):  # All done
                print(b.score)
                print(b)
            
    if len(boards) == len(boards_done):
        break

27.. 14.. 70.. 7.. 85.. 66.. 65.. 57.. 68.. 23.. 33.. 78.. 4.. 84.. 25.. 18.. 43.. 71.. 76.. 61.. 34.. 82.. 93.. 74.. 26.. 15.. 83.. 64.. 2.. 35.. 19.. 97.. 32.. 47.. 6.. 51.. 99.. 20.. 77.. 75.. 56.. 73.. 80.. 86.. 55.. 36.. 13.. 95.. 52.. 63.. 79.. 72.. 9.. 10.. 16.. 8.. 69.. 11.. 50.. 54.. 81.. 22.. 45.. 1.. 12.. 88.. 44.. 17.. 62.. 0.. 96.. 94.. 31.. 90.. 39.. 92.. 37.. 40.. 5.. 98.. 24.. 38.. 46.. 21.. 30.. 49.. 41.. 12833
*52	*31	*24	*68	*41
 48	*82	*19	 29	*65
*51	 91	*97	*39	*80
 3	*55	*43	*40	*38
*20	 89	 53	*45	*75
